In [ ]:
import os
import rasterio
import cv2 as cv
import sys
import pandas as pd

sys.path.insert(1, "../")
from utils import *

* ECC works for images of the same type with very good overlap. not working with images with small overlaps (not coorectly registering for multi-sensor cases at least without pre-processing)
* very very slow! (for original images)
* Phase correlation is basically global co-registration in AROSICS. not working for transformation. It only does translation shifts but it is very fast. It can support multi-sensor as it could work on images with partial overlaps.
* It seems that optical flow is doing better than phase correlation but the improvement is trivial in multi-sensor. It does much better in optical pairs. Also it is very fast.

In [ ]:
DATA_DIR = "../data/files_list"

S1_PRODUCTS = ["SLC", "GRD"]
S2_PRODUCTS = ["L1C", "L2A"]

s1_au_df = pd.read_csv(os.path.join(DATA_DIR, "s1_au.csv"), names=["ID", "Path"])
s1_an_df = pd.read_csv(os.path.join(DATA_DIR, "s1_an.csv"), names=["ID", "Path"])

s2_au_df = pd.read_csv(os.path.join(DATA_DIR, "s2_au.csv"), names=["ID", "Path"])
s2_an_df = pd.read_csv(os.path.join(DATA_DIR, "s2_an.csv"), names=["ID", "Path"])

s1_au_slc_dict = get_scenes_dict(s1_au_df, ["SLC"])
s1_au_grd_dict = get_scenes_dict(s1_au_df, ["GRD"])

s1_an_slc_dict = get_scenes_dict(s1_an_df, ["SLC"])
s1_an_grd_dict = get_scenes_dict(s1_an_df, ["GRD"])

s2_au_l1c_dict = get_scenes_dict(s2_au_df, ["L1C"], False)
s2_au_l2a_dict = get_scenes_dict(s2_au_df, ["L2A"], False)

s2_an_l1c_dict = get_scenes_dict(s2_an_df, ["L1C"], False)
s2_an_l2a_dict = get_scenes_dict(s2_an_df, ["L2A"], False)

#### Coregister arbitrary scenes

In [ ]:
s2_an_l1c_r031_dict = get_scenes_dict(s2_an_df, ["L1C", "R031"], False)
id = list(s2_an_l1c_r031_dict.keys())[0]
tdf = s2_an_df[s2_an_df.ID == id]
tdf = tdf[tdf.Path.apply(lambda x: "L1C" in x)]
tdf = tdf[tdf.Path.apply(lambda x: "N0511_R031" in x)]
s2_an_l1c_secne_files = list(tdf.Path)
ref_id = 4
ref_scene = s2_an_l1c_secne_files[ref_id]

shutil.rmtree(f"../data/inputs/{id}/", ignore_errors=True)
with ZipFile(ref_scene) as f:
    f.extractall(f"../data/inputs/{id}/ref")
ref_image_dir = os.path.join("../data/inputs/", id, "ref")

ref_tci_files = list(
    filter(
        lambda f: "TCI" in f,
        glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True),
    )
)
ref_tci_files = list(filter(lambda f: "L1C" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

print(f"Reference image: {ref_image}")

params = {
    "translation_x": 5.0,
    "translation_y": 10.0,
    "rotation_angle": 2.5,
    "scale": 1.0,
}

tgt_image_dir = os.path.join("../data/inputs/", id, "tgt")
os.makedirs(tgt_image_dir, exist_ok=True)

tgt_image = os.path.join(tgt_image_dir, "tgt.tif")
warp_affine_dataset(ref_image, tgt_image, **params)

_, (axb, axt) = plt.subplots(1, 2, figsize=(10, 20))
show(downsample_dataset(ref_image, 0.1)[0], ax=axb, title="Reference scene")
show(downsample_dataset(tgt_image, 0.1)[0], ax=axt, title="Target scene")

In [ ]:
ref_image_ds = os.path.join(ref_image_dir, "ref_ds.tif")
downsample_dataset(ref_image, 0.1, ref_image_ds)
tgt_image_ds = os.path.join(tgt_image_dir, "tgt_ds.tif")
_ = downsample_dataset(tgt_image, 0.1, tgt_image_ds)

In [ ]:
ref_img = cv.cvtColor(
    flip_img(rasterio.open(ref_image).read().copy()), cv.COLOR_BGR2GRAY
)
tgt_img = cv.cvtColor(
    flip_img(rasterio.open(tgt_image).read().copy()), cv.COLOR_BGR2GRAY
)
ref_img_ds = cv.cvtColor(
    flip_img(rasterio.open(ref_image_ds).read().copy()), cv.COLOR_BGR2GRAY
)
tgt_img_ds = cv.cvtColor(
    flip_img(rasterio.open(tgt_image_ds).read().copy()), cv.COLOR_BGR2GRAY
)

##### ECC

In [ ]:
warp_mode = cv.MOTION_HOMOGRAPHY
warp_matrix = np.eye(3, 3, dtype=np.float32)
number_of_iterations = 5000
termination_eps = 1e-5

criteria = (
    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
    number_of_iterations,
    termination_eps,
)
(cc, warp_matrix) = cv.findTransformECC(
    ref_img_ds, tgt_img_ds, warp_matrix, warp_mode, criteria
)

tgt_aligned_homography_ds = cv.warpPerspective(
    tgt_img_ds,
    warp_matrix,
    (tgt_img_ds.shape[1], tgt_img_ds.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

# tgt_aligned_homography = cv.warpPerspective(
#     tgt_img,
#     warp_matrix,
#     (tgt_img.shape[1], tgt_img.shape[0]),
#     flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
# )

In [ ]:
warp_mode = cv.MOTION_AFFINE
warp_matrix = np.eye(2, 3, dtype=np.float32)
number_of_iterations = 5000
termination_eps = 1e-5

criteria = (
    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
    number_of_iterations,
    termination_eps,
)
(cc, warp_matrix) = cv.findTransformECC(
    ref_img_ds, tgt_img_ds, warp_matrix, warp_mode, criteria
)

tgt_aligned_affine_ds = cv.warpAffine(
    tgt_img_ds,
    warp_matrix,
    (tgt_img_ds.shape[1], tgt_img_ds.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

# tgt_aligned_affine = cv.warpAffine(
#     tgt_img,
#     warp_matrix,
#     (tgt_img.shape[1], tgt_img.shape[0]),
#     flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
# )

In [ ]:
_, axes = plt.subplots(1, 4, figsize=(15, 10))
axes[0].imshow(ref_img_ds, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img_ds, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img_ds, tgt_img_ds), 3)}")
axes[2].imshow(tgt_aligned_homography_ds, cmap="jet")
axes[2].set_xlabel(
    f"Registered (hompgraphy), ssim:{np.round(ssim(ref_img_ds, tgt_aligned_homography_ds), 3)}"
)
axes[3].imshow(tgt_aligned_affine_ds, cmap="jet")
axes[3].set_xlabel(
    f"Registered (affine), ssim:{np.round(ssim(ref_img_ds, tgt_aligned_affine_ds), 3)}"
)

In [ ]:
# _, axes = plt.subplots(1, 4, figsize=(15, 10))
# axes[0].imshow(ref_img, cmap="jet")
# axes[0].set_xlabel("Reference")
# axes[1].imshow(tgt_img, cmap="jet")
# axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img, tgt_img), 3)}")
# axes[2].imshow(tgt_aligned_homography, cmap="jet")
# axes[2].set_xlabel(
#     f"Registered (hompgraphy), ssim:{np.round(ssim(ref_img, tgt_aligned_homography), 3)}"
# )
# axes[3].imshow(tgt_aligned_affine, cmap="jet")
# axes[3].set_xlabel(
#     f"Registered (affine), ssim:{np.round(ssim(ref_img, tgt_aligned_affine), 3)}"
# )

##### Phase Correlation

In [ ]:
window = cv.createHanningWindow(ref_img_ds.shape, cv.CV_32F)
# window = None
((shift_x, shift_y), phase_corr_score) = cv.phaseCorrelate(
    np.float32(tgt_img_ds), np.float32(ref_img_ds), window
)
tgt_shifted_ds = warp_affine_dataset(
    tgt_img_ds, translation_x=shift_x, translation_y=shift_y
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img_ds, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img_ds, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img_ds, tgt_img_ds), 3)}")
axes[2].imshow(tgt_shifted_ds, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img_ds, tgt_shifted_ds), 3)}")

In [ ]:
# window = cv.createHanningWindow(ref_img.shape, cv.CV_32F)
window = None
((shift_x, shift_y), phase_corr_score) = cv.phaseCorrelate(
    np.float32(tgt_img), np.float32(ref_img), window
)
tgt_shifted = warp_affine_dataset(tgt_img, translation_x=shift_x, translation_y=shift_y)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img, tgt_img), 3)}")
axes[2].imshow(tgt_shifted, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img, tgt_shifted), 3)}")

##### Optical flow

In [ ]:
# params for ShiTomasi corner detection
feature_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7,
)

# Parameters for lucas kanade optical flow
number_of_iterations = 10
termination_eps = 1e-5
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(
        cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
        number_of_iterations,
        termination_eps,
    ),
)

In [ ]:
p0 = cv.goodFeaturesToTrack(ref_img_ds, mask=None, **feature_params)
p1, st, err = cv.calcOpticalFlowPyrLK(ref_img_ds, tgt_img_ds, p0, None, **lk_params)
ref_good = p0[st == 1].astype(np.float32)
tgt_good = p1[st == 1].astype(np.float32)
warp_matrix = cv.findHomography(ref_good, tgt_good, cv.RANSAC)[0]
tgt_aligned_ds = cv.warpPerspective(
    tgt_img_ds,
    warp_matrix,
    (tgt_img_ds.shape[1], tgt_img_ds.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img_ds, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img_ds, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img_ds, tgt_img_ds), 3)}")
axes[2].imshow(tgt_aligned_ds, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img_ds, tgt_aligned_ds), 3)}")

In [ ]:
# p0 = cv.goodFeaturesToTrack(ref_img, mask=None, **feature_params)
# p1, st, err = cv.calcOpticalFlowPyrLK(ref_img, tgt_img, p0, None, **lk_params)
# ref_good = p0[st == 1].astype(np.float32)
# tgt_good = p1[st == 1].astype(np.float32)
# warp_matrix = cv.findHomography(ref_good, tgt_good, cv.RANSAC)[0]
# tgt_aligned = cv.warpPerspective(
#     tgt_img,
#     warp_matrix,
#     (tgt_img.shape[1], tgt_img.shape[0]),
#     flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
# )

In [ ]:
# _, axes = plt.subplots(1, 3, figsize=(15, 10))
# axes[0].imshow(ref_img, cmap="jet")
# axes[0].set_xlabel("Reference")
# axes[1].imshow(tgt_img, cmap="jet")
# axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img, tgt_img), 3)}")
# axes[2].imshow(tgt_aligned, cmap="jet")
# axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img, tgt_aligned), 3)}")

##### Feature matching

In [ ]:
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15
orb = cv.ORB_create(MAX_FEATURES)
keypoints1, descriptors1 = orb.detectAndCompute(ref_img_ds, None)
keypoints2, descriptors2 = orb.detectAndCompute(tgt_img_ds, None)

matcher = cv.DescriptorMatcher_create(cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
matches = matcher.match(descriptors1, descriptors2, None)

matches = sorted(matches, key=lambda x: x.distance)

numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
matches = matches[:numGoodMatches]

imMatches = cv.drawMatches(
    ref_img_ds, keypoints1, tgt_img_ds, keypoints2, matches, None
)
cv.imwrite("matches.jpg", imMatches)

# Extract location of good matches
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

# Find homography
h, mask = cv.findHomography(points1, points2, cv.RANSAC)

# Use homography
height, width = ref_img_ds.shape
tgt_aligned_ds = cv.warpPerspective(
    tgt_img_ds, h, (width, height), flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img_ds, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img_ds, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img_ds, tgt_img_ds), 3)}")
axes[2].imshow(tgt_aligned_ds, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img_ds, tgt_aligned_ds), 3)}")

In [ ]:
# MAX_FEATURES = 500
# GOOD_MATCH_PERCENT = 0.15
# orb = cv.ORB_create(MAX_FEATURES)
# keypoints1, descriptors1 = orb.detectAndCompute(ref_img, None)
# keypoints2, descriptors2 = orb.detectAndCompute(tgt_img, None)

# matcher = cv.DescriptorMatcher_create(cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
# matches = matcher.match(descriptors1, descriptors2, None)

# matches = sorted(matches, key=lambda x:x.distance)

# numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
# matches = matches[:numGoodMatches]

# imMatches = cv.drawMatches(ref_img, keypoints1, tgt_img, keypoints2, matches, None)
# cv.imwrite("matches.jpg", imMatches)

# # Extract location of good matches
# points1 = np.zeros((len(matches), 2), dtype=np.float32)
# points2 = np.zeros((len(matches), 2), dtype=np.float32)

# for i, match in enumerate(matches):
#     points1[i, :] = keypoints1[match.queryIdx].pt
#     points2[i, :] = keypoints2[match.trainIdx].pt

# # Find homography
# h, mask = cv.findHomography(points1, points2, cv.RANSAC)

# # Use homography
# height, width = ref_img.shape
# tgt_aligned = cv.warpPerspective(tgt_img, h, (width, height), flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP)

In [ ]:
# _, axes = plt.subplots(1, 3, figsize=(15, 10))
# axes[0].imshow(ref_img, cmap="jet")
# axes[0].set_xlabel("Reference")
# axes[1].imshow(tgt_img, cmap="jet")
# axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img, tgt_img), 3)}")
# axes[2].imshow(tgt_aligned, cmap="jet")
# axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img, tgt_aligned), 3)}")

#### Coregister S1 SLC (RTC) and S2 L1C 

In [ ]:
ref_scene = "../data/outputs/AROSICS/L1C_SLC_AU_pair/ref.tif"
tgt_scene = "../data/outputs/AROSICS/L1C_SLC_AU_pair/tgt.tif"

downsample_dataset(ref_scene, 0.1, "../data/outputs/OpenCV/L1C_SLC_AU/ref_ds.tif")
_ = downsample_dataset(tgt_scene, 0.1, "../data/outputs/OpenCV/L1C_SLC_AU/tgt_ds.tif")

In [ ]:
ref_scene = "../data/outputs/OpenCV/L1C_SLC_AU/ref_ds.tif"
tgt_scene = "../data/outputs/OpenCV/L1C_SLC_AU/tgt_ds.tif"

In [ ]:
coords, warps = find_overlap(ref_scene, tgt_scene, True, True)

In [ ]:
ref_img = cv.cvtColor(
    flip_img(rasterio.open(ref_scene).read().copy()), cv.COLOR_BGR2GRAY
)
tgt_img = cv.cvtColor(
    flip_img(rasterio.open(tgt_scene).read().copy()), cv.COLOR_BGR2GRAY
)

ref_warp = cv.cvtColor(warps[2], cv.COLOR_BGR2GRAY)
tgt_warp = cv.cvtColor(warps[3], cv.COLOR_BGR2GRAY)

##### ECC

In [ ]:
warp_mode = cv.MOTION_HOMOGRAPHY
warp_matrix = np.eye(3, 3, dtype=np.float32)
number_of_iterations = 5000
termination_eps = 1e-5

criteria = (
    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
    number_of_iterations,
    termination_eps,
)
(cc, warp_matrix) = cv.findTransformECC(
    ref_warp, tgt_warp, warp_matrix, warp_mode, criteria
)

tgt_aligned_homography_ds = cv.warpPerspective(
    tgt_warp,
    warp_matrix,
    (tgt_warp.shape[1], tgt_warp.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [ ]:
warp_mode = cv.MOTION_AFFINE
warp_matrix = np.eye(2, 3, dtype=np.float32)
number_of_iterations = 5000
termination_eps = 1e-5

criteria = (
    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
    number_of_iterations,
    termination_eps,
)
(cc, warp_matrix) = cv.findTransformECC(
    ref_warp, tgt_warp, warp_matrix, warp_mode, criteria
)

tgt_aligned_affine_ds = cv.warpAffine(
    tgt_warp,
    warp_matrix,
    (tgt_warp.shape[1], tgt_warp.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(simple_mosaic([ref_warp, tgt_warp]), cmap="gray")
axes[0].set_xlabel(f"Original, ssim:{np.round(ssim(ref_warp, tgt_warp), 3)}")
axes[1].imshow(simple_mosaic([ref_warp, tgt_aligned_homography_ds]), cmap="gray")
axes[1].set_xlabel(
    f"Registered (hompgraphy), ssim:{np.round(ssim(ref_warp, tgt_aligned_homography_ds), 3)}"
)
axes[2].imshow(simple_mosaic([ref_warp, tgt_aligned_affine_ds]), cmap="gray")
axes[2].set_xlabel(
    f"Registered (affine), ssim:{np.round(ssim(ref_warp, tgt_aligned_affine_ds), 3)}"
)

##### Phase correlation

In [ ]:
# window = cv.createHanningWindow((ref_warp.shape[1], ref_warp.shape[0]), cv.CV_32F)
window = None
((shift_x, shift_y), phase_corr_score) = cv.phaseCorrelate(
    np.float32(tgt_warp), np.float32(ref_warp), window
)
tgt_shifted = warp_affine_dataset(
    tgt_warp, translation_x=shift_x, translation_y=shift_y
)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(15, 10))
axes[0].imshow(simple_mosaic([ref_warp, tgt_warp]), cmap="gray")
axes[0].set_xlabel(
    f"Original, ssim:{np.round(ssim(ref_warp, tgt_warp), 3)}, mse:{np.round(mse(ref_warp, tgt_warp), 3)}"
)
axes[1].imshow(simple_mosaic([ref_warp, tgt_shifted]), cmap="gray")
axes[1].set_xlabel(
    f"Registered, ssim:{np.round(ssim(ref_warp, tgt_shifted), 3)}, mse:{np.round(mse(ref_warp, tgt_shifted), 3)}"
)

##### Optical Flow

In [ ]:
# params for ShiTomasi corner detection
feature_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7,
)

# Parameters for lucas kanade optical flow
number_of_iterations = 10
termination_eps = 1e-5
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(
        cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
        number_of_iterations,
        termination_eps,
    ),
)

In [ ]:
p0 = cv.goodFeaturesToTrack(ref_warp, mask=None, **feature_params)
p1, st, err = cv.calcOpticalFlowPyrLK(ref_warp, tgt_warp, p0, None, **lk_params)
ref_good = p0[st == 1].astype(np.float32)
tgt_good = p1[st == 1].astype(np.float32)
warp_matrix = cv.findHomography(ref_good, tgt_good)[0]
tgt_aligned_ds = cv.warpPerspective(
    tgt_warp,
    warp_matrix,
    (tgt_warp.shape[1], tgt_warp.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(15, 10))
axes[0].imshow(simple_mosaic([ref_warp, tgt_warp]), cmap="gray")
axes[0].set_xlabel(f"Original, ssim:{np.round(ssim(ref_warp, tgt_warp), 3)}")
axes[1].imshow(simple_mosaic([ref_warp, tgt_aligned_ds]), cmap="gray")
axes[1].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_warp, tgt_aligned_ds), 3)}")

##### Feature matching

In [ ]:
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15
orb = cv.ORB_create(MAX_FEATURES)
keypoints1, descriptors1 = orb.detectAndCompute(ref_warp, None)
keypoints2, descriptors2 = orb.detectAndCompute(tgt_warp, None)

matcher = cv.DescriptorMatcher_create(cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
matches = matcher.match(descriptors1, descriptors2, None)

matches = sorted(matches, key=lambda x: x.distance)

numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
matches = matches[:numGoodMatches]

imMatches = cv.drawMatches(ref_warp, keypoints1, tgt_warp, keypoints2, matches, None)
cv.imwrite("matches.jpg", imMatches)

# Extract location of good matches
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

# Find homography
h, mask = cv.findHomography(points1, points2, cv.RANSAC)

# Use homography
height, width = ref_warp.shape
tgt_aligned_ds = cv.warpPerspective(
    tgt_warp, h, (width, height), flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP
)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(15, 10))
axes[0].imshow(simple_mosaic([ref_warp, tgt_warp]), cmap="gray")
axes[0].set_xlabel(f"Original, ssim:{np.round(ssim(ref_warp, tgt_warp), 3)}")
axes[1].imshow(simple_mosaic([ref_warp, tgt_aligned_ds]), cmap="gray")
axes[1].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_warp, tgt_aligned_ds), 3)}")

##### Test two S2 L2A images for AN

In [ ]:
id = list(s2_an_l2a_dict.keys())[1]
s2_an_l2a_secne_files = s2_an_l2a_dict[id]

# get the Jan 2023 and Dec 2024 files
s2_an_l2a_secne_pair = [s2_an_l2a_secne_files[10], s2_an_l2a_secne_files[-1]]

shutil.rmtree(f"../data/inputs/{id}/", ignore_errors=True)
for i, zip_file_path in enumerate(s2_an_l2a_secne_pair):
    with ZipFile(zip_file_path) as f:
        f.extractall(f"../data/inputs/{id}/sub{i}")
ref_image_dir = os.path.join("../data/inputs/", id, f"sub{0}")
ref_tci_files = list(
    filter(
        lambda f: "TCI" in f,
        glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True),
    )
)
ref_tci_files = list(filter(lambda f: "L2A" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

tgt_image_dir = os.path.join("../data/inputs/", id, f"sub{1}")
tgt_tci_files = list(
    filter(
        lambda f: "TCI" in f,
        glob.glob(f"{tgt_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True),
    )
)
tgt_tci_files = list(filter(lambda f: "L2A" in f, tgt_tci_files))
if len(tgt_tci_files) > 1:
    tgt_image = [f for f in tgt_tci_files if f.endswith("_10m.jp2")][0]
else:
    tgt_image = tgt_tci_files[0]

_, (axb, axt) = plt.subplots(1, 2, figsize=(10, 20))
show(downsample_dataset(ref_image, 0.1)[0], ax=axb, title="Reference scene")
show(downsample_dataset(tgt_image, 0.1)[0], ax=axt, title="Target scene")

In [ ]:
ref_image_ds = os.path.join(ref_image_dir, "ref_ds.tif")
downsample_dataset(ref_image, 0.1, ref_image_ds)
tgt_image_ds = os.path.join(tgt_image_dir, "tgt_ds.tif")
_ = downsample_dataset(tgt_image, 0.1, tgt_image_ds)

In [ ]:
ref_img = cv.cvtColor(
    flip_img(rasterio.open(ref_image).read().copy()), cv.COLOR_BGR2GRAY
)
tgt_img = cv.cvtColor(
    flip_img(rasterio.open(tgt_image).read().copy()), cv.COLOR_BGR2GRAY
)
ref_img_ds = cv.cvtColor(
    flip_img(rasterio.open(ref_image_ds).read().copy()), cv.COLOR_BGR2GRAY
)
tgt_img_ds = cv.cvtColor(
    flip_img(rasterio.open(tgt_image_ds).read().copy()), cv.COLOR_BGR2GRAY
)

##### ECC

In [ ]:
warp_mode = cv.MOTION_HOMOGRAPHY
warp_matrix = np.eye(3, 3, dtype=np.float32)
number_of_iterations = 5000
termination_eps = 1e-5

criteria = (
    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
    number_of_iterations,
    termination_eps,
)
(cc, warp_matrix) = cv.findTransformECC(
    ref_img_ds, tgt_img_ds, warp_matrix, warp_mode, criteria
)

tgt_aligned_homography_ds = cv.warpPerspective(
    tgt_img_ds,
    warp_matrix,
    (tgt_img_ds.shape[1], tgt_img_ds.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

# tgt_aligned_homography = cv.warpPerspective(
#     tgt_img,
#     warp_matrix,
#     (tgt_img.shape[1], tgt_img.shape[0]),
#     flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
# )

In [ ]:
warp_mode = cv.MOTION_AFFINE
warp_matrix = np.eye(2, 3, dtype=np.float32)
number_of_iterations = 5000
termination_eps = 1e-5

criteria = (
    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
    number_of_iterations,
    termination_eps,
)
(cc, warp_matrix) = cv.findTransformECC(
    ref_img_ds, tgt_img_ds, warp_matrix, warp_mode, criteria
)

tgt_aligned_affine_ds = cv.warpAffine(
    tgt_img_ds,
    warp_matrix,
    (tgt_img_ds.shape[1], tgt_img_ds.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

# tgt_aligned_affine = cv.warpAffine(
#     tgt_img,
#     warp_matrix,
#     (tgt_img.shape[1], tgt_img.shape[0]),
#     flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
# )

In [ ]:
_, axes = plt.subplots(1, 4, figsize=(15, 10))
axes[0].imshow(ref_img_ds, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img_ds, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img_ds, tgt_img_ds), 3)}")
axes[2].imshow(tgt_aligned_homography_ds, cmap="jet")
axes[2].set_xlabel(
    f"Registered (hompgraphy), ssim:{np.round(ssim(ref_img_ds, tgt_aligned_homography_ds), 3)}"
)
axes[3].imshow(tgt_aligned_affine_ds, cmap="jet")
axes[3].set_xlabel(
    f"Registered (affine), ssim:{np.round(ssim(ref_img_ds, tgt_aligned_affine_ds), 3)}"
)

In [ ]:
# _, axes = plt.subplots(1, 4, figsize=(15, 10))
# axes[0].imshow(ref_img, cmap="jet")
# axes[0].set_xlabel("Reference")
# axes[1].imshow(tgt_img, cmap="jet")
# axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img, tgt_img), 3)}")
# axes[2].imshow(tgt_aligned_homography, cmap="jet")
# axes[2].set_xlabel(
#     f"Registered (hompgraphy), ssim:{np.round(ssim(ref_img, tgt_aligned_homography), 3)}"
# )
# axes[3].imshow(tgt_aligned_affine, cmap="jet")
# axes[3].set_xlabel(
#     f"Registered (affine), ssim:{np.round(ssim(ref_img, tgt_aligned_affine), 3)}"
# )

##### phase correlation

In [ ]:
window = cv.createHanningWindow(ref_img_ds.shape, cv.CV_32F)
# window = None
((shift_x, shift_y), phase_corr_score) = cv.phaseCorrelate(
    np.float32(tgt_img_ds), np.float32(ref_img_ds), window
)
tgt_shifted_ds = warp_affine_dataset(
    tgt_img_ds, translation_x=shift_x, translation_y=shift_y
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img_ds, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img_ds, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img_ds, tgt_img_ds), 3)}")
axes[2].imshow(tgt_shifted_ds, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img_ds, tgt_shifted_ds), 3)}")

In [ ]:
# window = cv.createHanningWindow(ref_img.shape, cv.CV_32F)
window = None
((shift_x, shift_y), phase_corr_score) = cv.phaseCorrelate(
    np.float32(tgt_img), np.float32(ref_img), window
)
tgt_shifted = warp_affine_dataset(tgt_img, translation_x=shift_x, translation_y=shift_y)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img, tgt_img), 3)}")
axes[2].imshow(tgt_shifted, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img, tgt_shifted), 3)}")

##### optical flow

In [ ]:
# params for ShiTomasi corner detection
feature_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7,
)

# Parameters for lucas kanade optical flow
number_of_iterations = 10
termination_eps = 1e-5
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(
        cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
        number_of_iterations,
        termination_eps,
    ),
)

In [ ]:
p0 = cv.goodFeaturesToTrack(ref_img_ds, mask=None, **feature_params)
p1, st, err = cv.calcOpticalFlowPyrLK(ref_img_ds, tgt_img_ds, p0, None, **lk_params)
ref_good = p0[st == 1].astype(np.float32)
tgt_good = p1[st == 1].astype(np.float32)
warp_matrix = cv.findHomography(ref_good, tgt_good, cv.RANSAC)[0]
tgt_aligned_ds = cv.warpPerspective(
    tgt_img_ds,
    warp_matrix,
    (tgt_img_ds.shape[1], tgt_img_ds.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img_ds, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img_ds, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img_ds, tgt_img_ds), 3)}")
axes[2].imshow(tgt_aligned_ds, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img_ds, tgt_aligned_ds), 3)}")

##### feature matching

In [ ]:
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15
orb = cv.ORB_create(MAX_FEATURES)
keypoints1, descriptors1 = orb.detectAndCompute(ref_img_ds, None)
keypoints2, descriptors2 = orb.detectAndCompute(tgt_img_ds, None)

matcher = cv.DescriptorMatcher_create(cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
matches = matcher.match(descriptors1, descriptors2, None)

matches = sorted(matches, key=lambda x: x.distance)

numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
matches = matches[:numGoodMatches]

imMatches = cv.drawMatches(
    ref_img_ds, keypoints1, tgt_img_ds, keypoints2, matches, None
)
cv.imwrite("matches.jpg", imMatches)

# Extract location of good matches
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

# Find homography
# h, mask = cv.findHomography(points1, points2, cv.RANSAC)
h, _ = cv.estimateAffine2D(points1, points2)

# Use homography
height, width = ref_img_ds.shape
# tgt_aligned_ds = cv.warpPerspective(
tgt_aligned_ds = cv.warpAffine(
    tgt_img_ds, h, (width, height), flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP
)

In [ ]:
_, axes = plt.subplots(1, 3, figsize=(15, 10))
axes[0].imshow(ref_img_ds, cmap="jet")
axes[0].set_xlabel("Reference")
axes[1].imshow(tgt_img_ds, cmap="jet")
axes[1].set_xlabel(f"Target, ssim:{np.round(ssim(ref_img_ds, tgt_img_ds), 3)}")
axes[2].imshow(tgt_aligned_ds, cmap="jet")
axes[2].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_img_ds, tgt_aligned_ds), 3)}")

##### S1 RTC (SLC) Pair

In [ ]:
ref_scene = "../data/asf/SLC_Pair/S1A_IW_20230130T192218_DVP_RTC20_G_gpufed_B6D4.zip"
tgt_scene = "../data/asf/SLC_Pair/S1A_IW_20241209T191428_DVP_RTC20_G_gpufed_8B70.zip"
shutil.rmtree(f"../data/inputs/25S150E-30S155E/", ignore_errors=True)

with ZipFile(ref_scene) as f:
    f.extractall(f"../data/inputs/25S150E-30S155E/ref")
ref_image_dir = os.path.join("../data/inputs/", "25S150E-30S155E", "ref")

with ZipFile(tgt_scene) as f:
    f.extractall(f"../data/inputs/25S150E-30S155E/tgt")
tgt_image_dir = os.path.join("../data/inputs/", "25S150E-30S155E", "tgt")

ref_image = list(
    filter(
        lambda f: ("_rgb" in f) and f.endswith(".tif"),
        glob.glob(f"{ref_image_dir}/*/*"),
    )
)[0]
tgt_image = list(
    filter(
        lambda f: ("_rgb" in f) and f.endswith(".tif"),
        glob.glob(f"{tgt_image_dir}/*/*"),
    )
)[0]

_, (axb, axt) = plt.subplots(1, 2, figsize=(10, 20))
show(downsample_dataset(ref_image, 0.1)[0], ax=axb, title="Reference scene")
show(downsample_dataset(tgt_image, 0.1)[0], ax=axt, title="Target scene")

In [ ]:
coords, warps = find_overlap(ref_image, tgt_image, True, True)

In [ ]:
ref_img = cv.cvtColor(
    flip_img(rasterio.open(ref_image).read().copy()), cv.COLOR_BGR2GRAY
)
tgt_img = cv.cvtColor(
    flip_img(rasterio.open(tgt_image).read().copy()), cv.COLOR_BGR2GRAY
)

ref_warp = cv.cvtColor(warps[2], cv.COLOR_BGR2GRAY)
tgt_warp = cv.cvtColor(warps[3], cv.COLOR_BGR2GRAY)

##### ECC

Does not work

##### Phase correlation

In [ ]:
# window = cv.createHanningWindow((ref_warp.shape[1], ref_warp.shape[0]), cv.CV_32F)
window = None
((shift_x, shift_y), phase_corr_score) = cv.phaseCorrelate(
    np.float32(tgt_warp), np.float32(ref_warp), window
)
tgt_shifted = warp_affine_dataset(
    tgt_warp, translation_x=shift_x, translation_y=shift_y
)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(15, 10))
axes[0].imshow(simple_mosaic([ref_warp, tgt_warp]), cmap="gray")
axes[0].set_xlabel(f"Original, ssim:{np.round(ssim(ref_warp, tgt_warp), 3)}")
axes[1].imshow(simple_mosaic([ref_warp, tgt_shifted]), cmap="gray")
axes[1].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_warp, tgt_shifted), 3)}")

##### optical flow

In [ ]:
# params for ShiTomasi corner detection
feature_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7,
)

# Parameters for lucas kanade optical flow
number_of_iterations = 10
termination_eps = 1e-5
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(
        cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
        number_of_iterations,
        termination_eps,
    ),
)

In [ ]:
p0 = cv.goodFeaturesToTrack(ref_warp, mask=None, **feature_params)
p1, st, err = cv.calcOpticalFlowPyrLK(ref_warp, tgt_warp, p0, None, **lk_params)
ref_good = p0[st == 1].astype(np.float32)
tgt_good = p1[st == 1].astype(np.float32)
warp_matrix = cv.findHomography(ref_good, tgt_good)[0]
tgt_aligned_ds = cv.warpPerspective(
    tgt_warp,
    warp_matrix,
    (tgt_warp.shape[1], tgt_warp.shape[0]),
    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(15, 10))
axes[0].imshow(simple_mosaic([ref_warp, tgt_warp]), cmap="gray")
axes[0].set_xlabel(f"Original, ssim:{np.round(ssim(ref_warp, tgt_warp), 3)}")
axes[1].imshow(simple_mosaic([ref_warp, tgt_aligned_ds]), cmap="gray")
axes[1].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_warp, tgt_aligned_ds), 3)}")

##### feature matching

In [ ]:
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15
orb = cv.ORB_create(MAX_FEATURES)
keypoints1, descriptors1 = orb.detectAndCompute(ref_warp, None)
keypoints2, descriptors2 = orb.detectAndCompute(tgt_warp, None)

matcher = cv.DescriptorMatcher_create(cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
matches = matcher.match(descriptors1, descriptors2, None)

matches = sorted(matches, key=lambda x: x.distance)

numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
matches = matches[:numGoodMatches]

imMatches = cv.drawMatches(ref_warp, keypoints1, tgt_warp, keypoints2, matches, None)
cv.imwrite("matches.jpg", imMatches)

# Extract location of good matches
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

# Find homography
h, mask = cv.findHomography(points1, points2, cv.RANSAC)

# Use homography
height, width = ref_warp.shape
tgt_aligned_ds = cv.warpPerspective(
    tgt_warp, h, (width, height), flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP
)

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(15, 10))
axes[0].imshow(simple_mosaic([ref_warp, tgt_warp]), cmap="gray")
axes[0].set_xlabel(f"Original, ssim:{np.round(ssim(ref_warp, tgt_warp), 3)}")
axes[1].imshow(simple_mosaic([ref_warp, tgt_aligned_ds]), cmap="gray")
axes[1].set_xlabel(f"Registered, ssim:{np.round(ssim(ref_warp, tgt_aligned_ds), 3)}")

##### S2 AN Series

In [ ]:
s2_an_l1c_r031_dict = get_scenes_dict(s2_an_df, ["L1C", "R031"], False)
id = list(s2_an_l1c_r031_dict.keys())[0]

tdf = s2_an_df[s2_an_df.ID == id]
tdf = tdf[tdf.Path.apply(lambda x: "L1C" in x)]
tdf = tdf[tdf.Path.apply(lambda x: "N0511_R031" in x)]
s2_an_l1c_secne_files = list(tdf.Path)

shutil.rmtree(f"../data/inputs/{id}/", ignore_errors=True)

ref_id = 4
ref_scene = s2_an_l1c_secne_files[ref_id]
with ZipFile(ref_scene) as f:
    f.extractall(f"../data/inputs/{id}/ref")
ref_image_dir = os.path.join("../data/inputs/", id, "ref")
ref_tci_files = list(
    filter(
        lambda f: "TCI" in f,
        glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True),
    )
)
ref_tci_files = list(filter(lambda f: "L1C" in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

downsample_dataset(ref_image, 0.1, ref_image.replace(".jp2", "_ds.jp2"))
ref_image = ref_image.replace(".jp2", "_ds.jp2")

tgt_scenes = [
    s2_an_l1c_secne_files[i]
    for i in range(0, len(s2_an_l1c_secne_files))
    if i != ref_id
]
tgt_images = []
local_outputs = []
global_outputs = []
for i, zip_file_path in enumerate(tgt_scenes):
    print(f"Extracting {i + 1} of {len(tgt_scenes)}: {zip_file_path}")
    with ZipFile(zip_file_path) as f:
        f.extractall(f"../data/inputs/{id}/tgt{i}")
    tgt_image_dir = os.path.join("../data/inputs/", id, f"tgt{i}")
    tgt_tci_files = list(
        filter(
            lambda f: "TCI" in f,
            glob.glob(f"{tgt_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True),
        )
    )
    tgt_tci_files = list(filter(lambda f: "L1C" in f, tgt_tci_files))
    if len(tgt_tci_files) > 1:
        tgt_image = [f for f in tgt_tci_files if f.endswith("_10m.jp2")][0]
    else:
        tgt_image = tgt_tci_files[0]
    tgt_images.append(tgt_image)

for img in tgt_images:
    downsample_dataset(img, 0.1, img.replace(".jp2", "_ds.jp2"))
tgt_images = [img.replace(".jp2", "_ds.jp2") for img in tgt_images]

In [ ]:
ref_img = cv.cvtColor(
    flip_img(rasterio.open(ref_image).read().copy()), cv.COLOR_BGR2GRAY
)
tgt_imgs = [
    cv.cvtColor(flip_img(rasterio.open(tgt_image).read().copy()), cv.COLOR_BGR2GRAY)
    for tgt_image in tgt_images
]

##### ECC

In [ ]:
warp_mode = cv.MOTION_AFFINE
number_of_iterations = 5000
termination_eps = 1e-5

criteria = (
    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
    number_of_iterations,
    termination_eps,
)

tgt_aligned_list = []
processed_tgt_images = []
processed_output_images = []

temp_dir = "../data/outputs/OpenCV/AN_S2_Series/temp"
os.makedirs(temp_dir, exist_ok=True)
for i, tgt_img in enumerate(tgt_imgs):
    try:
        warp_matrix = np.eye(2, 3, dtype=np.float32)
        (cc, warp_matrix) = cv.findTransformECC(
            ref_img, tgt_img, warp_matrix, warp_mode, criteria
        )

        tgt_aligned = cv.warpAffine(
            tgt_img,
            warp_matrix,
            (tgt_img.shape[1], tgt_img.shape[0]),
            flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
        )

        tgt_aligned_list.append(tgt_aligned)

        output_path = os.path.join(temp_dir, f"out_{i}.tiff")
        processed_output_images.append(output_path)
        processed_tgt_images.append(tgt_images[i])

        profile = rasterio.open(tgt_images[i]).profile
        with rasterio.open(output_path, "w", **profile) as ds:
            for i in range(0, profile["count"]):
                ds.write(tgt_aligned, i + 1)
    except:
        print(f"Algorithm did not converge for {tgt_images[i]}")
        tgt_aligned_list.append(np.zeros(0))

In [ ]:
out_gif = "../data/outputs/OpenCV/AN_S2_Series/ecc.gif"
fids = [
    int(os.path.splitext(os.path.basename(tgt))[0].split("_")[-1])
    for tgt in processed_output_images
]
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] + processed_output_images
datasets_titles = ["Reference"] + [
    f"target_{id}, ssim:{np.round(ssim(ref_img, tgt_aligned_list[id]), 3)}"
    for id in fids
]
make_difference_gif(datasets_paths, out_gif, datasets_titles)

out_gif = "../data/outputs/OpenCV/AN_S2_Series/raw_ecc.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] + processed_tgt_images
datasets_titles = ["Reference"] + [
    f"target_{id}, ssim:{np.round(ssim(ref_img, tgt_imgs[id]), 3)}" for id in fids
]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
shutil.rmtree(temp_dir, ignore_errors=True)

##### Phase correlation

In [ ]:
tgt_aligned_list = []
processed_tgt_images = []
processed_output_images = []

temp_dir = "../data/outputs/OpenCV/AN_S2_Series/temp"
os.makedirs(temp_dir, exist_ok=True)
for i, tgt_img in enumerate(tgt_imgs):
    try:
        # window = cv.createHanningWindow(ref_img.shape, cv.CV_32F)
        window = None
        ((shift_x, shift_y), phase_corr_score) = cv.phaseCorrelate(
            np.float32(tgt_img), np.float32(ref_img), window
        )
        tgt_aligned = warp_affine_dataset(
            tgt_img, translation_x=shift_x, translation_y=shift_y
        )

        tgt_aligned_list.append(tgt_aligned)

        output_path = os.path.join(temp_dir, f"out_{i}.tiff")
        processed_output_images.append(output_path)
        processed_tgt_images.append(tgt_images[i])

        profile = rasterio.open(tgt_images[i]).profile
        with rasterio.open(output_path, "w", **profile) as ds:
            for i in range(0, profile["count"]):
                ds.write(tgt_aligned, i + 1)
    except:
        print(f"Algorithm did not converge for {tgt_images[i]}")
        tgt_aligned_list.append(np.zeros(0))

In [ ]:
out_gif = "../data/outputs/OpenCV/AN_S2_Series/pc.gif"
fids = [
    int(os.path.splitext(os.path.basename(tgt))[0].split("_")[-1])
    for tgt in processed_output_images
]
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] + processed_output_images
datasets_titles = ["Reference"] + [
    f"target_{id}, ssim:{np.round(ssim(ref_img, tgt_aligned_list[id]), 3)}"
    for id in fids
]
make_difference_gif(datasets_paths, out_gif, datasets_titles)

out_gif = "../data/outputs/OpenCV/AN_S2_Series/raw_pc.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] + processed_tgt_images
datasets_titles = ["Reference"] + [
    f"target_{id}, ssim:{np.round(ssim(ref_img, tgt_imgs[id]), 3)}" for id in fids
]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
shutil.rmtree(temp_dir, ignore_errors=True)

##### optical flow

In [ ]:
# params for ShiTomasi corner detection
feature_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7,
)

# Parameters for lucas kanade optical flow
number_of_iterations = 10
termination_eps = 1e-5
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(
        cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
        number_of_iterations,
        termination_eps,
    ),
)

In [ ]:
tgt_aligned_list = []
processed_tgt_images = []
processed_output_images = []

temp_dir = "../data/outputs/OpenCV/AN_S2_Series/temp"
os.makedirs(temp_dir, exist_ok=True)
for i, tgt_img in enumerate(tgt_imgs):
    try:
        p0 = cv.goodFeaturesToTrack(ref_img, mask=None, **feature_params)
        p1, st, err = cv.calcOpticalFlowPyrLK(ref_img, tgt_img, p0, None, **lk_params)
        ref_good = p0[st == 1].astype(np.float32)
        tgt_good = p1[st == 1].astype(np.float32)
        warp_matrix = cv.findHomography(ref_good, tgt_good, cv.RANSAC)[0]
        tgt_aligned = cv.warpPerspective(
            tgt_img,
            warp_matrix,
            (tgt_img.shape[1], tgt_img.shape[0]),
            flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
        )

        tgt_aligned_list.append(tgt_aligned)

        output_path = os.path.join(temp_dir, f"out_{i}.tiff")
        processed_output_images.append(output_path)
        processed_tgt_images.append(tgt_images[i])

        profile = rasterio.open(tgt_images[i]).profile
        with rasterio.open(output_path, "w", **profile) as ds:
            for i in range(0, profile["count"]):
                ds.write(tgt_aligned, i + 1)
    except:
        print(f"Algorithm did not converge for {tgt_images[i]}")
        tgt_aligned_list.append(np.zeros(0))

In [ ]:
out_gif = "../data/outputs/OpenCV/AN_S2_Series/of.gif"
fids = [
    int(os.path.splitext(os.path.basename(tgt))[0].split("_")[-1])
    for tgt in processed_output_images
]
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] + processed_output_images
datasets_titles = ["Reference"] + [
    f"target_{id}, ssim:{np.round(ssim(ref_img, tgt_aligned_list[id]), 3)}"
    for id in fids
]
make_difference_gif(datasets_paths, out_gif, datasets_titles)

out_gif = "../data/outputs/OpenCV/AN_S2_Series/raw_of.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] + processed_tgt_images
datasets_titles = ["Reference"] + [
    f"target_{id}, ssim:{np.round(ssim(ref_img, tgt_imgs[id]), 3)}" for id in fids
]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
shutil.rmtree(temp_dir, ignore_errors=True)

##### feature matching

In [ ]:
MAX_FEATURES = 500
GOOD_MATCH_PERCENT = 0.15

tgt_aligned_list = []
processed_tgt_images = []
processed_output_images = []

temp_dir = "../data/outputs/OpenCV/AN_S2_Series/temp"
os.makedirs(temp_dir, exist_ok=True)
for i, tgt_img in enumerate(tgt_imgs):
    try:
        orb = cv.ORB_create(MAX_FEATURES)
        keypoints1, descriptors1 = orb.detectAndCompute(ref_img, None)
        keypoints2, descriptors2 = orb.detectAndCompute(tgt_img, None)

        matcher = cv.DescriptorMatcher_create(cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
        matches = matcher.match(descriptors1, descriptors2, None)

        matches = sorted(matches, key=lambda x: x.distance)

        numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
        matches = matches[:numGoodMatches]

        # Extract location of good matches
        points1 = np.zeros((len(matches), 2), dtype=np.float32)
        points2 = np.zeros((len(matches), 2), dtype=np.float32)

        for j, match in enumerate(matches):
            points1[j, :] = keypoints1[match.queryIdx].pt
            points2[j, :] = keypoints2[match.trainIdx].pt

        # Find homography
        h, mask = cv.findHomography(points1, points2, cv.RANSAC)

        # Use homography
        height, width = ref_img.shape
        tgt_aligned = cv.warpPerspective(
            tgt_img, h, (width, height), flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP
        )

        tgt_aligned_list.append(tgt_aligned)

        output_path = os.path.join(temp_dir, f"out_{i}.tiff")
        processed_output_images.append(output_path)
        processed_tgt_images.append(tgt_images[i])

        profile = rasterio.open(tgt_images[i]).profile
        with rasterio.open(output_path, "w", **profile) as ds:
            for i in range(0, profile["count"]):
                ds.write(tgt_aligned, i + 1)
    except:
        print(f"Algorithm did not converge for {tgt_images[i]}")
        tgt_aligned_list.append(np.zeros(0))

In [ ]:
out_gif = "../data/outputs/OpenCV/AN_S2_Series/orb.gif"
fids = [
    int(os.path.splitext(os.path.basename(tgt))[0].split("_")[-1])
    for tgt in processed_output_images
]
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] + processed_output_images
datasets_titles = ["Reference"] + [
    f"target_{id}, ssim:{np.round(ssim(ref_img, tgt_aligned_list[id]), 3)}"
    for id in fids
]
make_difference_gif(datasets_paths, out_gif, datasets_titles)

out_gif = "../data/outputs/OpenCV/AN_S2_Series/raw_orb.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] + processed_tgt_images
datasets_titles = ["Reference"] + [
    f"target_{id}, ssim:{np.round(ssim(ref_img, tgt_imgs[id]), 3)}" for id in fids
]
make_difference_gif(datasets_paths, out_gif, datasets_titles)
shutil.rmtree(temp_dir, ignore_errors=True)

##### S2 L1C Series

In [ ]:
Tile = "T42DWK"
original_scenes = glob.glob(f"../data/inputs/{Tile}/L1C/**")
ds_scenes_dir = f"../data/outputs/AROSICS/L1C_AN_Series_{Tile}_DS"
os.makedirs(ds_scenes_dir, exist_ok=True)
# for sc in original_scenes:
#     downsample_dataset(sc, 0.1, os.path.join(ds_scenes_dir, os.path.basename(sc)))
downsampled_scenes = [
    fn for fn in glob.glob(f"{ds_scenes_dir}/**") if fn.endswith(".jp2")
]
ref_image = downsampled_scenes[1]
tgt_images = [downsampled_scenes[0]] + downsampled_scenes[2:]

ref_img = cv.cvtColor(
    flip_img(rasterio.open(ref_image).read().copy()), cv.COLOR_BGR2GRAY
)
tgt_imgs = [
    cv.cvtColor(flip_img(rasterio.open(tgt_image).read().copy()), cv.COLOR_BGR2GRAY)
    for tgt_image in tgt_images
]

In [ ]:
coreg_method = "of"

match coreg_method:
    case "ecc":
        warp_mode = cv.MOTION_AFFINE
        number_of_iterations = 5000
        termination_eps = 1e-5

        params_dic = dict(
            criteria=(
                cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
                number_of_iterations,
                termination_eps,
            )
        )
    case "pc":
        params_dic = dict(
            # window = cv.createHanningWindow(ref_img.shape, cv.CV_32F)
            window=None
        )
    case "of":
        number_of_iterations = 10
        termination_eps = 1e-5

        params_dic = dict(
            # params for ShiTomasi corner detection
            feature_params=dict(
                maxCorners=100,
                qualityLevel=0.3,
                minDistance=7,
                blockSize=7,
            ),
            # Parameters for lucas kanade optical flow
            lk_params=dict(
                winSize=(15, 15),
                maxLevel=2,
                criteria=(
                    cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT,
                    number_of_iterations,
                    termination_eps,
                ),
            ),
            err_threshold=25,
        )
    case "orb":
        params_dic = dict(
            max_features=500,
            good_match_percent=0.15,
        )

print("Coreg method: " + coreg_method)
print("Params dict:")
print(params_dic)

tgt_aligned_list = []
processed_tgt_images = []
processed_output_images = []

temp_dir = f"../data/outputs/OpenCV/{Tile}_AN_S2_Series/temp"
os.makedirs(temp_dir, exist_ok=True)
for i, tgt_img in enumerate(tgt_imgs):
    try:
        match coreg_method:
            case "ecc":
                warp_matrix = np.eye(2, 3, dtype=np.float32)
                (cc, warp_matrix) = cv.findTransformECC(
                    ref_img, tgt_img, warp_matrix, warp_mode, params_dic["criteria"]
                )

                tgt_aligned = cv.warpAffine(
                    tgt_img,
                    warp_matrix,
                    (tgt_img.shape[1], tgt_img.shape[0]),
                    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
                )
            case "pc":
                ((shift_x, shift_y), phase_corr_score) = cv.phaseCorrelate(
                    np.float32(tgt_img), np.float32(ref_img), params_dic["window"]
                )
                tgt_aligned = warp_affine_dataset(
                    tgt_img, translation_x=shift_x, translation_y=shift_y
                )
            case "of":
                p0 = cv.goodFeaturesToTrack(
                    ref_img, mask=None, **params_dic["feature_params"]
                )
                p1, st, err = cv.calcOpticalFlowPyrLK(
                    ref_img, tgt_img, p0, None, **params_dic["lk_params"]
                )
                # ref_good = p0[st == 1].astype(np.float32)
                # tgt_good = p1[st == 1].astype(np.float32)
                dist = np.linalg.norm(p1[st == 1] - p0[st == 1], axis=1)
                of_idx = np.where(np.squeeze(dist) < params_dic["err_threshold"])
                ref_good = np.squeeze(p0)[of_idx].astype("int")
                tgt_good = np.squeeze(p1)[of_idx].astype("int")
                valid_idx = np.all(
                    (
                        tgt_good[:, 0] >= 0,
                        tgt_good[:, 1] >= 0,
                        tgt_good[:, 0] < tgt_img.shape[1],
                        tgt_good[:, 1] < tgt_img.shape[0],
                    ),
                    axis=0,
                )
                tgt_good = np.expand_dims(tgt_good[valid_idx], axis=0)
                ref_good = np.expand_dims(ref_good[valid_idx], axis=0)
                if (ref_good.shape[1] < 4) or (tgt_good.shape[1] < 4):
                    print(
                        f"WARNING: couldn't find enough good features for target or reference. num ref features: {ref_good.shape[0]}, num tgt features = {tgt_good.shape[0]}"
                    )

                warp_matrix = cv.findHomography(ref_good, tgt_good, cv.RANSAC)[0]
                tgt_aligned = cv.warpPerspective(
                    tgt_img,
                    warp_matrix,
                    (tgt_img.shape[1], tgt_img.shape[0]),
                    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
                )
            case "orb":
                orb = cv.ORB_create(params_dic["max_features"])
                keypoints1, descriptors1 = orb.detectAndCompute(ref_img, None)
                keypoints2, descriptors2 = orb.detectAndCompute(tgt_img, None)

                matcher = cv.DescriptorMatcher_create(
                    cv.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING
                )
                matches = matcher.match(descriptors1, descriptors2, None)

                matches = sorted(matches, key=lambda x: x.distance)

                numGoodMatches = int(len(matches) * params_dic["good_match_percent"])
                matches = matches[:numGoodMatches]

                # Extract location of good matches
                points1 = np.zeros((len(matches), 2), dtype=np.float32)
                points2 = np.zeros((len(matches), 2), dtype=np.float32)

                for j, match in enumerate(matches):
                    points1[j, :] = keypoints1[match.queryIdx].pt
                    points2[j, :] = keypoints2[match.trainIdx].pt

                # Find homography
                # h, mask = cv.findHomography(points1, points2, cv.RANSAC)
                h, _ = cv.estimateAffine2D(points1, points2)

                # Use homography
                height, width = ref_img.shape
                # tgt_aligned = cv.warpPerspective(
                tgt_aligned = cv.warpAffine(
                    tgt_img,
                    h,
                    (width, height),
                    flags=cv.INTER_LINEAR + cv.WARP_INVERSE_MAP,
                )

        tgt_aligned_list.append(tgt_aligned)

        output_path = os.path.join(temp_dir, f"out_{i}.tiff")
        processed_output_images.append(output_path)
        processed_tgt_images.append(tgt_images[i])

        profile = rasterio.open(tgt_images[i]).profile
        with rasterio.open(output_path, "w", **profile) as ds:
            for i in range(0, profile["count"]):
                ds.write(tgt_aligned, i + 1)
    except:
        print(f"Algorithm did not converge for {tgt_images[i]}")
        tgt_aligned_list.append(np.zeros(0))

In [ ]:
out_gif = f"../data/outputs/OpenCV/{Tile}_AN_S2_Series/{coreg_method}.gif"
fids = [
    int(os.path.splitext(os.path.basename(tgt))[0].split("_")[-1])
    for tgt in processed_output_images
]
target_titles = [f"target_{id}" for id in fids]

if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] + processed_output_images
ssims_aligned = [np.round(ssim(ref_img, tgt_aligned_list[id]), 3) for id in fids]
mse_aligned = [np.round(mse(ref_img, tgt_aligned_list[id]), 3) for id in fids]
datasets_titles = ["Reference"] + [
    f"{target_title}, ssim:{ssim_score}, mse:{mse_score}"
    for target_title, ssim_score, mse_score in zip(
        target_titles, ssims_aligned, mse_aligned
    )
]
make_difference_gif(datasets_paths, out_gif, datasets_titles)

out_gif = f"../data/outputs/OpenCV/{Tile}_AN_S2_Series/raw_{coreg_method}.gif"
if os.path.isfile(out_gif):
    os.remove(out_gif)
datasets_paths = [ref_image] + processed_tgt_images
ssims_raw = [np.round(ssim(ref_img, tgt_imgs[id]), 3) for id in fids]
mse_raw = [np.round(mse(ref_img, tgt_imgs[id]), 3) for id in fids]
datasets_titles = ["Reference"] + [
    f"{target_title}, ssim:{ssim_score}, mse:{mse_score}"
    for target_title, ssim_score, mse_score in zip(target_titles, ssims_raw, mse_raw)
]
make_difference_gif(datasets_paths, out_gif, datasets_titles)

out_ssim = f"../data/outputs/OpenCV/{Tile}_AN_S2_Series/{coreg_method}.csv"
out_ssim_df = pd.DataFrame(
    zip(target_titles, ssims_raw, ssims_aligned, mse_raw, mse_aligned),
    columns=["Title", "SSIM Raw", "SSIM Aligned", "MSE Raw", "MSE Aligned"],
    index=None,
)
out_ssim_df.to_csv(out_ssim, encoding="utf-8")

shutil.rmtree(temp_dir, ignore_errors=True)

##### S2 L1C Series AN (T42CVE) (SIF/OF with PC)

In [ ]:
import os
import rasterio
import cv2 as cv
import sys
import pandas as pd

sys.path.insert(1, "../")
from utils import *

In [ ]:
Tile = "T42DWK"
output_path = f"../data/outputs/OpenCV/{Tile}_AN_S2_Series_SIF-OF_PC"

original_scenes = glob.glob(f"../data/inputs/{Tile}/L1C/**")
ds_scenes_dir = f"../data/outputs/AROSICS/L1C_AN_Series_{Tile}_DS"
os.makedirs(ds_scenes_dir, exist_ok=True)
# for sc in original_scenes:
#     downsample_dataset(sc, 0.1, os.path.join(ds_scenes_dir, os.path.basename(sc)))
downsampled_scenes = [
    fn for fn in glob.glob(f"{ds_scenes_dir}/**") if fn.endswith(".jp2")
]
ref_image = downsampled_scenes[1]
tgt_images = [downsampled_scenes[0]] + downsampled_scenes[2:]

In [ ]:
number_of_iterations = 10
termination_eps = 1e-5
of_params: dict = dict(
    # params for ShiTomasi corner detection
    feature_params=dict(
        maxCorners=1000,
        qualityLevel=0.3,
        minDistance=7,
        blockSize=7,
    ),
    # Parameters for lucas kanade optical flow
    lk_params=dict(
        winSize=(55, 55),
        maxLevel=2,
    ),
)
_, shifts = co_register(
    ref_image,
    tgt_images,
    filtering_mode="of",
    output_path=output_path,
    of_params=of_params,
    number_of_iterations=number_of_iterations,
    termination_eps=termination_eps,
    enhanced_shift_method="corr",
    remove_outlilers=True,
    corr_thresh=0.1,
)

#### Landsat Series

In [ ]:
output_path = f"../data/outputs/OpenCV/L1GT_127111_Original"
# output_path = f"../data/outputs/OpenCV/LANDSAT_8_108074"
original_scenes = glob.glob(f"../data/inputs/landsat/true_color/**")
# original_scenes = glob.glob(f"../data/inputs/LANDSAT_8_108074/true_color/**")
# downsampled_scenes = glob.glob(f"../data/inputs/landsat/true_color_ds/**")
ref_image = original_scenes[0]
tgt_images = original_scenes[1:]

In [ ]:
_, shifts = co_register(
    ref_image,
    tgt_images,
    output_path=output_path,
    remove_outlilers=True,
    # laplacian_kernel_size = 5,
    return_shifted_images=True,
    # rethrow_error=True,
    use_overlap=True,
)

In [ ]:
shifts

##### S1 RTC Series

In [ ]:
output_path = "../data/outputs/OpenCV/RTC_Series"
enhanced_scenes = [
    f for f in glob.glob("../data/outputs/AROSICS/RTC_Series/**") if f.endswith(".tif")
]
ref_image = enhanced_scenes[0]
tgt_images = enhanced_scenes[1:]

In [ ]:
number_of_iterations = 10
termination_eps = 1e-5
of_params: dict = dict(
    # params for ShiTomasi corner detection
    feature_params=dict(
        maxCorners=1000,
        qualityLevel=0.3,
        minDistance=7,
        blockSize=7,
    ),
    # Parameters for lucas kanade optical flow
    lk_params=dict(
        winSize=(55, 55),
        maxLevel=2,
    ),
)
_, shifts = co_register(
    ref_image,
    tgt_images,
    filtering_mode="pca_of",
    output_path=output_path,
    of_params=of_params,
    number_of_iterations=number_of_iterations,
    termination_eps=termination_eps,
    enhanced_shift_method="mean",
    remove_outlilers=True,
    corr_thresh=0.1,
)